In [ ]:
import glob
from PIL import Image
from torch.autograd import Variable
from os import listdir
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler    
import torchvision
from torchvision import datasets,transforms 
import boto3
from io import BytesIO 
import pickle
import requests
import json
from datetime import datetime
from pytz import timezone


image_name = "k"

bucket = "attendance-model"
file_key = "model_final"

def read_joblib(path):
    ''' 
       Function to load a joblib file from an s3 bucket or local directory.
       Arguments:
       * path: an s3 bucket or local directory path where the file is stored
       Outputs:
       * file: Joblib file loaded
    '''

    # Path is an s3 bucket
    if path[:5] == 's3://':
        s3_bucket, s3_key = path.split('/')[2], path.split('/')[3:]
        s3_key = '/'.join(s3_key)
        with BytesIO() as f:
            boto3.client("s3").download_fileobj(Bucket=s3_bucket, Key=s3_key, Fileobj=f)
            f.seek(0)
            file = torch.load(f)
    
    # Path is a local directory 
    else:
        with open(path, 'rb') as f:
            file = torch.load(f)
    
    return file


def imshow(inp, title):

    inp = inp.cpu().numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    
    plt.figure (figsize = (12, 6))

    plt.imshow(inp)
    plt.title(title)
    plt.pause(2)
    
with torch.no_grad():    
    model_path = read_joblib('s3://attendance-model/model_final')
    correct = 0
    total = 0
    array = [['1131901','Durvesh'], ['1149695','Frances'] ,['1146105','Jay'], ['1138948','Kamal'], ['1138765','yash']]
    
    mean = [0.485, 0.456, 0.406]
    std  = [0.229, 0.224, 0.225]


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean,
                             std=std)
    ])
    #onlyfiles = [f for f in listdir(base_path)]
    #print(onlyfiles)
    img = Image.open("./"+image_name+".jpg")
    trans = transforms.ToPILImage()
    trans1 = transforms.ToTensor()
    transformed_image = test_transform(img).float()
    transformed_image = Variable(transformed_image, requires_grad=True)
    transformed_image = transformed_image.unsqueeze(0)
    #         plt.imshow(transformed_image)
    #         print(transformed_image)
    transformed_image = transformed_image.to(device)
    outputs = model_path(transformed_image)
    _, predicted = torch.max(outputs.data, 1)
    predicted_index_value = predicted.cpu().numpy()[0]
    print("Predicted image is : " +str(predicted_index_value))
    imshow(trans1(img),array[predicted_index_value])
    

api_get = 'https://1o739743u5.execute-api.us-east-1.amazonaws.com/getV1/putdyanamodb?id='+array[predicted_index_value][0]

headers_dict = {"id" : "1138948", "Content-Type": "application/json",
                     "Accept": "application/json"}   

res = requests.get(api_get,headers=headers_dict)
json_data = res.json()


api_post = "https://1o739743u5.execute-api.us-east-1.amazonaws.com/v1/putdyanamodb"


array_Count = json_data["Count"]



fmt = '%Y-%m-%d %H:%M:%S'
eastern = timezone('US/Eastern')
loc_dt = datetime.now(eastern)

current_date = loc_dt.strftime(fmt)
print(current_date)

att = [
        str(current_date)
      ]

str1 = ','.join(str(e) for e in att)
print(str1)




if array_Count == 1:
    s = json_data["Items"][0]
    password = s["password"]["S"]
    last_name = s["last_name"]["S"]
    attendance = s["attendance"]["S"]
    first_name = s["first_name"]["S"]
    id = s["id"]["S"]
    email = s["email"]["S"]
    
    concat_attendance = attendance +','+ str1

    string = {
        "id" : id,
        "first_name" : first_name,
        "last_name" : last_name,
        "email" : email,
        "password" : password,

        "attendance" : concat_attendance
    }

    string = json.dumps(string)
    res = requests.post(api_post, data=string)
    print('Data Successfully Created ',res.text)

